In [46]:
import os
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error, r2_score
import hopsworks
from functions import utils
import json
import warnings
warnings.filterwarnings("ignore")

In [47]:
from dotenv import load_dotenv
load_dotenv()
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
os.environ["HOPSWORKS_API_KEY"] = HOPSWORKS_API_KEY

In [48]:
project = hopsworks.login()
fs = project.get_feature_store() 
# secrets = utils.secrets_api(project.name)

"""
city_secrets = json.loads(secrets.get_secret("city_secrets").value)
CITY = city_secrets["city"]
latitude = city_secrets["latitude"]
longitude = city_secrets["longitude"]

time_secrets = json.loads(secrets.get_secret("time_secrets").value)
earliest_bike_datetime = time_secrets["earliest_bike_datetime"]
last_bike_datetime = time_secrets["last_bike_datetime"]
"""

CITY = "dublin"
# latitude =
# longitude =

earliest_bike_datetime = "2023-08-05 12:56:53 UTC"
last_bike_datetime = "2025-01-06 15:06:11 UTC"

today = datetime.now()

print(type(fs))

2025-01-07 14:06:09,044 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-07 14:06:09,048 INFO: Initializing external client
2025-01-07 14:06:09,049 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-07 14:06:10,339 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207494
<class 'hsfs.feature_store.FeatureStore'>


## Creating the feature view

In [49]:
# Retrieve feature groups
bike_fg = fs.get_feature_group(
    name='bike_data',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_data',
    version=1,
)

bike_fg_data = bike_fg.read()
weather_fg_data = weather_fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (24.53s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.47s) 


In [51]:

bike_fg_data["datetime"] =  pd.to_datetime(bike_fg_data["datetime"]).dt.floor('H')
weather_fg_data["datetime"] =  pd.to_datetime(weather_fg_data["datetime"]).dt.floor('H')

#bike_fg["datetime"] =  pd.to_datetime(bike_fg["datetime"]).replace(minute=0, second=0, microsecond=0)
#weather_fg["datetime"] =  pd.to_datetime(weather_fg["datetime"]).replace(minute=0, second=0, microsecond=0)
bike_fg_data

,datetime,station,num_bikes_available
0,2024-08-30 12:00:00+00:00,HARCOURT_TERRACE,18.0
1,2024-12-14 04:00:00+00:00,HEUSTON_BRIDGE_(SOUTH),25.0
2,2024-12-19 15:00:00+00:00,HANOVER_QUAY_EAST,18.0
3,2024-02-04 07:00:00+00:00,PARNELL_SQUARE_NORTH,5.0
4,2024-11-28 04:00:00+00:00,HARCOURT_TERRACE,4.0
...,...,...,...
1048192,2023-09-18 11:00:00+00:00,JAMES_STREET_EAST,21.0
1048193,2024-04-03 23:00:00+00:00,HANOVER_QUAY_EAST,16.0
1048194,2023-12-08 08:00:00+00:00,HATCH_STREET,10.0
1048195,2023-11-03 14:00:00+00:00,HANOVER_QUAY,20.0


In [52]:
# Merge feature groups
data = pd.merge(bike_fg_data, weather_fg_data, on="datetime", how="left")
data = data.dropna()
data = data.set_index("datetime")
data


TypeError: Expected type `str`, got `<class 'tuple'>`. Features are accessible by name.

In [34]:
# print("bike_fg " + str(type(bike_fg)), bike_fg.show(10))
print(bike_fg.select_all().join(weather_fg.select_all()).show(10))
# print("weather_fg " +str(type(weather_fg)), weather_fg)
# print("selected_features " +str(type(selected_features)))
# print("select bike " + bike_fg.select(["num_bikes_available", "station"]))
# print("select weather " + weather_fg.select_all().show(10))
# print("join " + bike_fg.select(["num_bikes_available", "station"]).join(weather_fg.select_all(), on=['datetime']).show(10)) 

RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1207494/featurestores/query). Server response: 
HTTP code: 400, HTTP reason: Bad Request, body: b'{"errorCode":270213,"usrMsg":"Provided feature name datetime is ambiguous and exists in more than one feature group. Consider prepending the prefix specified in the join.","errorMsg":"Feature name is ambiguous."}', error code: 270213, error msg: Feature name is ambiguous., user msg: Provided feature name datetime is ambiguous and exists in more than one feature group. Consider prepending the prefix specified in the join.

In [9]:
feature_view = fs.get_or_create_feature_view(
    name='bike_weather_fw',
    description="weather features with bike availability as the target",
    version=1,
    labels=['num_bikes_available'],
    query=selected_features,
)


Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1207494/fs/1195126/fv/bike_weather_fw/version/1


In [10]:
start_date_test_data = "2024-10-15"
# Convert string to datetime object
test_start = datetime.strptime(start_date_test_data, "%Y-%m-%d")

In [11]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(
    test_size=0.2, 
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (57.27s) 


In [12]:
X_train

,station,datetime,temperature_2m,apparent_temperature,rain,snowfall,wind_speed_10m,daylight_duration,rain_sum,city
0,HERBERT_PLACE,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,GOLDEN_LANE,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
5,GRANTHAM_STREET,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
7,EXCISE_WALK,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
9,FENIAN_STREET,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...
1047166,MOLESWORTH_STREET,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1047167,SMITHFIELD,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1047168,MOUNT_STREET_LOWER,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1047169,WILTON_TERRACE_(PARK),NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
